# Enable Query Acceleration Service for Warehouses with Eligible Queries
The query acceleration service (QAS) can accelerate parts of the query workload in a warehouse. When it is enabled for a warehouse, it can improve overall warehouse performance by reducing the impact of outlier queries, which are queries that use more resources than the typical query. The query acceleration service does this by offloading portions of the query processing work to shared compute resources that are provided by the service.
    
    For more information, visit:  https://docs.snowflake.com/en/user-guide/query-acceleration-service#label-query-acceleration-eligible-queries.
    
    This app identifies warehouses that execute queries that are eligible for QAS, along with the option to enable QAS for each warehouse. This app will:
    - check the `QUERY_ACCELERATION_ELIGIBLE` account usage view for warehouses that execute queries that are eligible for QAS.
        - The user can toggle the minimum number of eligible queries to check for, along with the threshold of average execution time is eligible for the service
    - enable QAS for each selected warehouse (optional)
    - allow the user to execute an eligible query on a warehouse once QAS is enabled (optional)

# Prerequesites
    
    - The user executing this app, must have access to the `SNOWFLAKE` database.
    - The user's role must either be the warehouse owner, or have the `MANAGE WAREHOUSE` account-level privilge to enable QAS for a selected warehouse.


This Notedbook will drop a streamlit file at SIT_SOLUTIONS.QAS.CODE_STAGE and install the app SIT_SOLUTIONS.QAS.QAS_STREAMLIT.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"qas_eligible_warehouses","version":{{"major":1, "minor":0}},"attributes":"session_tag"}}'""").collect()

Create solutions toolbox DB and QAS schema

In [ ]:
CREATE DATABASE IF NOT EXISTS SIT_SOLUTIONS;
CREATE OR REPLACE SCHEMA SIT_SOLUTIONS.QAS;
CREATE OR REPLACE STAGE SIT_SOLUTIONS.QAS.CODE_STAGE;

In [ ]:
CREATE OR REPLACE PROCEDURE SIT_SOLUTIONS.QAS.PUT_TO_STAGE(STAGE VARCHAR,FILENAME VARCHAR, CONTENT VARCHAR)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION=3.8
PACKAGES=('snowflake-snowpark-python')
HANDLER='put_to_stage'
COMMENT='{"origin":"sf_sit","name":"dmm","version":{"major":1, "minor":0},"attributes":{"component":"dmm"}}'
AS $$
import io
import os

def put_to_stage(session, stage, filename, content):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)
    f = open(local_file, "w", encoding='utf-8')
    f.write(content)
    f.close()
    session.file.put(local_file, '@'+stage, auto_compress=False, overwrite=True)
    return "saved file "+filename+" in stage "+stage
$$;


In [ ]:
encoded_obj = """gASV6DcAAAAAAAB9lCiMBWZpbGVzlH2UjBdxdWVyeV9hY2Nfd2FyZWhvdXNlcy5weZRCrTcAACMg
SW1wb3J0IHB5dGhvbiBwYWNrYWdlcwppbXBvcnQganNvbgppbXBvcnQgbnVtcHkgYXMgbnAKaW1w
b3J0IHBhbmRhcyBhcyBwZAppbXBvcnQgcmUKaW1wb3J0IHN0cmVhbWxpdCBhcyBzdAojIGltcG9y
dCBzcWxwYXJzZQojIGZyb20gc3RfYWdncmlkIGltcG9ydCBBZ0dyaWQsIEdyaWRVcGRhdGVNb2Rl
LCBKc0NvZGUKIyBmcm9tIHN0X2FnZ3JpZC5ncmlkX29wdGlvbnNfYnVpbGRlciBpbXBvcnQgR3Jp
ZE9wdGlvbnNCdWlsZGVyCiMgaW1wb3J0IHNxbHBhcnNlCmZyb20gc25vd2ZsYWtlLnNub3dwYXJr
LmNvbnRleHQgaW1wb3J0IGdldF9hY3RpdmVfc2Vzc2lvbgpzdC5zZXRfcGFnZV9jb25maWcobGF5
b3V0PSJ3aWRlIikKc3QuaW1hZ2UoJ2h0dHBzOi8vcmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbS9T
bm93Zmxha2UtTGFicy9lbWVyZ2luZy1zb2x1dGlvbnMtdG9vbGJveC9tYWluL2Jhbm5lcl9lbWVy
Z2luZ19zb2x1dGlvbnNfdG9vbGJveC5wbmcnKQoKc3Quc3ViaGVhZGVyKCJFbmFibGUgUXVlcnkg
QWNjZWxlcmF0aW9uIFNlcnZpY2UgZm9yIFdhcmVob3VzZXMgd2l0aCBFbGlnaWJsZSBRdWVyaWVz
IikKc3QubWFya2Rvd24oIiIiICAgIAogICAgVGhlIHF1ZXJ5IGFjY2VsZXJhdGlvbiBzZXJ2aWNl
IChRQVMpIGNhbiBhY2NlbGVyYXRlIHBhcnRzIG9mIHRoZSBxdWVyeSB3b3JrbG9hZCBpbiBhIHdh
cmVob3VzZS4gV2hlbiBpdCBpcyBlbmFibGVkIGZvciBhIHdhcmVob3VzZSwgaXQgY2FuIGltcHJv
dmUgb3ZlcmFsbCB3YXJlaG91c2UgcGVyZm9ybWFuY2UgYnkgcmVkdWNpbmcgdGhlIGltcGFjdCBv
ZiBvdXRsaWVyIHF1ZXJpZXMsIHdoaWNoIGFyZSBxdWVyaWVzIHRoYXQgdXNlIG1vcmUgcmVzb3Vy
Y2VzIHRoYW4gdGhlIHR5cGljYWwgcXVlcnkuIFRoZSBxdWVyeSBhY2NlbGVyYXRpb24gc2Vydmlj
ZSBkb2VzIHRoaXMgYnkgb2ZmbG9hZGluZyBwb3J0aW9ucyBvZiB0aGUgcXVlcnkgcHJvY2Vzc2lu
ZyB3b3JrIHRvIHNoYXJlZCBjb21wdXRlIHJlc291cmNlcyB0aGF0IGFyZSBwcm92aWRlZCBieSB0
aGUgc2VydmljZS4KICAgIAogICAgRm9yIG1vcmUgaW5mb3JtYXRpb24sIHZpc2l0OiAgaHR0cHM6
Ly9kb2NzLnNub3dmbGFrZS5jb20vZW4vdXNlci1ndWlkZS9xdWVyeS1hY2NlbGVyYXRpb24tc2Vy
dmljZSNsYWJlbC1xdWVyeS1hY2NlbGVyYXRpb24tZWxpZ2libGUtcXVlcmllcy4KICAgIAogICAg
VGhpcyBub3RlYm9vayBpZGVudGlmaWVzIHdhcmVob3VzZXMgdGhhdCBleGVjdXRlIHF1ZXJpZXMg
dGhhdCBhcmUgZWxpZ2libGUgZm9yIFFBUywgYWxvbmcgd2l0aCB0aGUgb3B0aW9uIHRvIGVuYWJs
ZSBRQVMgZm9yIGVhY2ggd2FyZWhvdXNlLiBUaGlzIG5vdGVib29rIHdpbGw6CiAgICAtIGNoZWNr
IHRoZSBgUVVFUllfQUNDRUxFUkFUSU9OX0VMSUdJQkxFYCBhY2NvdW50IHVzYWdlIHZpZXcgZm9y
IHdhcmVob3VzZXMgdGhhdCBleGVjdXRlIHF1ZXJpZXMgdGhhdCBhcmUgZWxpZ2libGUgZm9yIFFB
Uy4KICAgICAgICAtIFRoZSB1c2VyIGNhbiB0b2dnbGUgdGhlIG1pbmltdW0gbnVtYmVyIG9mIGVs
aWdpYmxlIHF1ZXJpZXMgdG8gY2hlY2sgZm9yLCBhbG9uZyB3aXRoIHRoZSB0aHJlc2hvbGQgb2Yg
YXZlcmFnZSBleGVjdXRpb24gdGltZSBpcyBlbGlnaWJsZSBmb3IgdGhlIHNlcnZpY2UKICAgIC0g
ZW5hYmxlIFFBUyBmb3IgZWFjaCBzZWxlY3RlZCB3YXJlaG91c2UgKG9wdGlvbmFsKQogICAgLSBh
bGxvdyB0aGUgdXNlciB0byBleGVjdXRlIGFuIGVsaWdpYmxlIHF1ZXJ5IG9uIGEgd2FyZWhvdXNl
IG9uY2UgUUFTIGlzIGVuYWJsZWQgKG9wdGlvbmFsKSIiIikKc3Quc3ViaGVhZGVyKCJQcmVyZXF1
ZXNpdGVzIikKc3QubWFya2Rvd24oIiIiCiAgICAtIFRoZSB1c2VyIGV4ZWN1dGluZyB0aGlzIG5v
dGVib29rLCBtdXN0IGhhdmUgYWNjZXNzIHRvIHRoZSBgU05PV0ZMQUtFYCBkYXRhYmFzZS4KICAg
IC0gVGhlIHVzZXIncyByb2xlIG11c3QgZWl0aGVyIGJlIHRoZSB3YXJlaG91c2Ugb3duZXIsIG9y
IGhhdmUgdGhlIGBNQU5BR0UgV0FSRUhPVVNFYCBhY2NvdW50LWxldmVsIHByaXZpbGdlIHRvIGVu
YWJsZSBRQVMgZm9yIGEgc2VsZWN0ZWQgd2FyZWhvdXNlLiIiIikKIyBXcml0ZSBkaXJlY3RseSB0
byB0aGUgYXBwCnNlc3Npb24gPSBnZXRfYWN0aXZlX3Nlc3Npb24oKQoKZGVmIHBhZ2luYXRlX2Rh
dGEoZGYpOgkJCQoJcGFnaW5hdGlvbiA9IHN0LmVtcHR5KCkKCWJhdGNoX3NpemUgPSAyMCAgIyBT
ZXQgdGhlIG51bWJlciBvZiBpdGVtcyBwZXIgcGFnZQoKCWlmIGxlbihkZikgPiAwOgoJCWJvdHRv
bV9tZW51ID0gc3QuY29sdW1ucygoNCwgMSwgMSkpCgkJd2l0aCBib3R0b21fbWVudVsyXToKCQkJ
dG90YWxfcGFnZXMgPSAoCiAgICAgICAgICAgICAgICBpbnQobGVuKGRmKSAvIGJhdGNoX3NpemUp
IGlmIGludChsZW4oZGYpIC8gYmF0Y2hfc2l6ZSkgPiAwIGVsc2UgMQogICAgICAgICAgICApCgkJ
CWN1cnJlbnRfcGFnZSA9IHN0Lm51bWJlcl9pbnB1dCgKICAgICAgICAgICAgICAgICJQYWdlIiwg
bWluX3ZhbHVlPTEsIG1heF92YWx1ZT10b3RhbF9wYWdlcywgc3RlcD0xCiAgICAgICAgICAgICkK
CQl3aXRoIGJvdHRvbV9tZW51WzBdOgoJCQlzdC5tYXJrZG93bihmIlBhZ2UgKip7Y3VycmVudF9w
YWdlfSoqIG9mICoqe3RvdGFsX3BhZ2VzfSoqICIpCiAgICAKCQlwYWdlcyA9IHNwbGl0X2ZyYW1l
KGRmLCBiYXRjaF9zaXplKQoJCSNkeW5hbWljYWxseSBzZXQgZGYgaGVpZ2h0LCBiYXNlZCBvbiBu
dW1iZXIgb2Ygcm93cyBpbiBkYXRhIGZyYW1lCgkJcGFnaW5hdGlvbi5kYXRhZnJhbWUoZGF0YT1w
YWdlc1tjdXJyZW50X3BhZ2UgLSAxXSwgaGVpZ2h0PWludCgoKGxlbihkZikvYmF0Y2hfc2l6ZSkg
KyAxLjUpICogNjAgKyAzLjUpLCB1c2VfY29udGFpbmVyX3dpZHRoPVRydWUpCgllbHNlOgoJCXN0
LmNhcHRpb24oIk5vIHJlc3VsdHMgdG8gZGlzcGxheS4iKQoKQHN0LmNhY2hlX2RhdGEoc2hvd19z
cGlubmVyPUZhbHNlKQpkZWYgc3BsaXRfZnJhbWUoaW5wdXRfZGYsIHJvd3MpOgoJZGYgPSBbaW5w
dXRfZGYubG9jW2kgOiBpICsgcm93cyAtIDEsIDpdIGZvciBpIGluIHJhbmdlKDAsIGxlbihpbnB1
dF9kZiksIHJvd3MpXQoJcmV0dXJuIGRmCgpAc3QuY2FjaGVfZGF0YShzaG93X3NwaW5uZXI9RmFs
c2UpCmRlZiBxdWVyeV9jYWNoZShfc2Vzc2lvbiwgaW5wdXRfc3RtdCk6CglkZiA9IHBkLkRhdGFG
cmFtZShfc2Vzc2lvbi5zcWwoaW5wdXRfc3RtdCkuY29sbGVjdCgpKQoJcmV0dXJuIGRmCgpzdC5k
aXZpZGVyKCkKc3Quc3ViaGVhZGVyKCJTVEVQIDE6IFNob3cgd2FyZWhvdXNlcyB0aGF0IGhhdmUg
cXVlcmllcyBlbGlnaWJsZSBmb3IgcXVlcnkgYWNjZWxlcmF0aW9uIikKc3Qud3JpdGUoIlRoaXMg
c3RlcCBzaG93cyB0aGUgd2FyZWhvdXNlcyB0aGF0IGhhdmUgcXVlcmllcyB0aGF0IGFyZSBlbGln
aWJsZSBmb3IgcXVlcnkgYWNjZWxlcmF0aW9uLCBiYXNlZCBvbiB0aGUgY3JpdGVyaWEgc2V0IGJ5
IHRoZSB1c2VyIGJlbG93LiIpCiNzZWxlY3QgZGF0ZSByYW5nZQpxYV9lbGlnaWJsZV9kYXRlX3Jh
bmdlX2xpc3QgPSBbJ0Nob29zZSBhIERhdGUgUmFuZ2UnLCAnTGFzdCBkYXknLCAnTGFzdCA3IGRh
eXMnLCAnTGFzdCAyOCBkYXlzJywgJ0xhc3QgMyBtb250aHMnLCAnTGFzdCA2IG1vbnRocycsICdM
YXN0IDEyIG1vbnRocyddCnN0LnNlbGVjdGJveCgiU2VsZWN0IERhdGUgUmFuZ2U6IiwgcWFfZWxp
Z2libGVfZGF0ZV9yYW5nZV9saXN0LCBrZXk9InNiX3FhX2VsaWdpYmxlX3JhbmdlIikKCmRhdGVf
dGltZV9wYXJ0ID0gIiIKaW5jcmVtZW50ID0gIiIKZGZfcXVlcnlfaGlzdG9yeV9yYW5nZSA9IE5v
bmUKZGlzYWJsZV9nZXRfZWxpZ2libGVfd2hzX2ZsYWcgPSBUcnVlCmxpc3RfcWFzX2VsaWdpYmxl
X3docyA9IFsiQ2hvb3NlIFdhcmVob3VzZSJdCmRmX3Fhc19lbGlnaWJsZV93aHNfY2xtbnMgPSBb
J1dhcmVob3VzZSBOYW1lJwogICAgICAgICAgICAgICAgICAgICAgICAgICAgICwnIyBvZiBFbGln
aWJsZSBRdWVyaWVzJwogICAgICAgICAgICAgICAgICAgICAgICAgICAgICwnQXZlcmFnZSBEdXJh
dGlvbiAoc2VjKScKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAsJ0F2ZXJhZ2UgJSBvZiBR
dWVyeSBBdmFpbGFibGUgZm9yIFFBJwogICAgICAgICAgICAgICAgICAgICAgICAgICAgXQoKaWYg
c3Quc2Vzc2lvbl9zdGF0ZS5zYl9xYV9lbGlnaWJsZV9yYW5nZSA9PSAiTGFzdCBkYXkiOgogICAg
ZGF0ZV90aW1lX3BhcnQgPSAiaG91cnMiCiAgICBpbmNyZW1lbnQgPSAiMjQiCmVsaWYgc3Quc2Vz
c2lvbl9zdGF0ZS5zYl9xYV9lbGlnaWJsZV9yYW5nZSA9PSAiTGFzdCA3IGRheXMiOgogICAgZGF0
ZV90aW1lX3BhcnQgPSAiZGF5cyIKICAgIGluY3JlbWVudCA9ICI3IgplbGlmIHN0LnNlc3Npb25f
c3RhdGUuc2JfcWFfZWxpZ2libGVfcmFuZ2UgPT0gIkxhc3QgMjggZGF5cyI6CiAgICBkYXRlX3Rp
bWVfcGFydCA9ICJkYXlzIgogICAgaW5jcmVtZW50ID0gIjI4IgplbGlmIHN0LnNlc3Npb25fc3Rh
dGUuc2JfcWFfZWxpZ2libGVfcmFuZ2UgPT0gIkxhc3QgMyBtb250aHMiOgogICAgZGF0ZV90aW1l
X3BhcnQgPSAibW9udGhzIgogICAgaW5jcmVtZW50ID0gIjMiCmVsaWYgc3Quc2Vzc2lvbl9zdGF0
ZS5zYl9xYV9lbGlnaWJsZV9yYW5nZSA9PSAiTGFzdCA2IG1vbnRocyI6CiAgICBkYXRlX3RpbWVf
cGFydCA9ICJtb250aHMiCiAgICBpbmNyZW1lbnQgPSAiNiIKZWxpZiBzdC5zZXNzaW9uX3N0YXRl
LnNiX3FhX2VsaWdpYmxlX3JhbmdlID09ICJMYXN0IDEyIG1vbnRocyI6CiAgICBkYXRlX3RpbWVf
cGFydCA9ICJtb250aHMiCiAgICBpbmNyZW1lbnQgPSAiMTIiCgojc2V0IG1pbmltdW0gbnVtYmVy
IG9mIGVsaWdpYmxlIHF1ZXJpZXMKc3QubnVtYmVyX2lucHV0KCJNaW5pbXVtICMgb2YgRWxpZ2li
bGUgUXVlcmllcyIsIG1pbl92YWx1ZT0xLCB2YWx1ZT0xLCBzdGVwPTEsIGtleT0ibnVtX21pbl9x
YXNfcXVlcmllcyIsIGhlbHA9IlRoZSBtaW5pbXVtIG51bWJlciBvZiBRQVMtZWxpZ2libGUgcXVl
cmllcyBmb3IgZWFjaCB3YXJlaG91c2UiKQoKI3NlbGVjdCBtaW5pbXVtIHJhdGlvIG9mIGVsaWdp
YmxlIHF1ZXJ5IGFjY2VsZXJhdGlvbiB0aW1lIHRvIHRvdGFsIHF1ZXJ5IGR1cmF0aW9uCnFhX2Vs
aWdpYmxlX3RpbWVfcGN0X2xpc3QgPSBbJ0Nob29zZSBhIFBlcmNlbnRhZ2UnLCAnMTAnLCAnMjUn
LCAnNTAnLCAnNzUnXQpzdC5zZWxlY3Rib3goIk1pbmltdW0gJSBvZiBRdWVyeSBFbGlnaWJsZSBm
b3IgUXVlcnkgQWNjZWxlcmF0aW9uOiIsIHFhX2VsaWdpYmxlX3RpbWVfcGN0X2xpc3QsIGtleT0i
c2JfcWFfZWxpZ2libGVfdGltZV9wY3QiLCBoZWxwPSJUaGUgbWluaW11bSBwZXJjZW50YWdlIG9m
IHRoZSBhbW91bnQgb2YgdGhlIHF1ZXJ5J3MgZXhlY3V0aW9uIHRpbWUgdGhhdCBpcyBlbGlnaWJs
ZSBmb3IgUUFTIikKCmlmIChzdC5zZXNzaW9uX3N0YXRlLnNiX3FhX2VsaWdpYmxlX3JhbmdlICAh
PSAiQ2hvb3NlIGEgRGF0ZSBSYW5nZSIpIGFuZCAoc3Quc2Vzc2lvbl9zdGF0ZS5zYl9xYV9lbGln
aWJsZV90aW1lX3BjdCAhPSAiQ2hvb3NlIGEgUGVyY2VudGFnZSIpOgogICAgZGlzYWJsZV9nZXRf
ZWxpZ2libGVfd2hzX2ZsYWcgPSBGYWxzZQoKc3QuYnV0dG9uKCJHZXQgRWxpZ2libGUgV2FyZWhv
dXNlcyIsIGRpc2FibGVkPWRpc2FibGVfZ2V0X2VsaWdpYmxlX3doc19mbGFnLCB0eXBlPSJwcmlt
YXJ5Iiwga2V5PSJidG5fZ2V0X2VsaWdpYmxlX3docyIpCgppZiBzdC5zZXNzaW9uX3N0YXRlLmJ0
bl9nZXRfZWxpZ2libGVfd2hzOgogICAgI2NyZWF0ZSBhIGRhdGFmcmFtZSBmcm9tIGVsaWdpYmxl
IHdhcmVob3VzZXMKICAgIGRmX3Fhc19lbGlnaWJsZV93aHMgPSBwZC5EYXRhRnJhbWUoc2Vzc2lv
bi5zcWwoZiIiIlNFTEVDVCAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgd2FyZWhvdXNlX25hbWUKICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgLENPVU5UKHF1ZXJ5X2lkKSBB
UyBudW1fZWxpZ2libGVfcXVlcmllcwogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAsQVZHKERBVEVESUZGKHNlY29uZCwgc3RhcnRfdGlt
ZSwgZW5kX3RpbWUpKTo6bnVtYmVyKDM4LDMpIEFTIGF2Z19kdXJhdGlvbl9zZWMKICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgLEFWRyhl
bGlnaWJsZV9xdWVyeV9hY2NlbGVyYXRpb25fdGltZSAvIE5VTExJRihEQVRFRElGRihzZWNvbmQs
IHN0YXJ0X3RpbWUsIGVuZF90aW1lKSwgMCkpOjpudW1iZXIoMzgsMykgKiAxMDAgYXMgYXZnX2Vs
aWdpYmxlX3RpbWVfcGN0CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgRlJPTSAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgU05PV0ZMQUtFLkFDQ09VTlRfVVNBR0UuUVVFUllfQUND
RUxFUkFUSU9OX0VMSUdJQkxFCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgV0hFUkUKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgZW5kX3RpbWUgPiBEQVRFQUREKHtkYXRlX3RpbWVf
cGFydH0sIC17aW5jcmVtZW50fSwgQ1VSUkVOVF9USU1FU1RBTVAoKSkKICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBHUk9VUCBCWSAKICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgd2Fy
ZWhvdXNlX25hbWUKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICBIQVZJTkcKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgbnVtX2VsaWdpYmxlX3F1ZXJpZXMgPj0ge3N0LnNlc3Npb25f
c3RhdGUubnVtX21pbl9xYXNfcXVlcmllc30KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgQU5EIGF2Z19lbGlnaWJsZV90aW1lX3BjdCA+
PSB7c3Quc2Vzc2lvbl9zdGF0ZS5zYl9xYV9lbGlnaWJsZV90aW1lX3BjdH0KICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBPUkRFUiBCWSAKICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
bnVtX2VsaWdpYmxlX3F1ZXJpZXMgREVTQyIiIikuY29sbGVjdCgpLCBjb2x1bW5zPWRmX3Fhc19l
bGlnaWJsZV93aHNfY2xtbnMpCgogICAgbGlzdF9xYXNfZWxpZ2libGVfd2hzLmV4dGVuZChkZl9x
YXNfZWxpZ2libGVfd2hzWydXYXJlaG91c2UgTmFtZSddLnRvbGlzdCgpKQogICAgc3Quc2Vzc2lv
bl9zdGF0ZS5saXN0X3Fhc19lbGlnaWJsZV93aHMgPSBsaXN0X3Fhc19lbGlnaWJsZV93aHMKCiAg
ICBzdC5kaXZpZGVyKCkKICAgIHN0LnN1YmhlYWRlcigiRWxpZ2libGUgV2FyZWhvdXNlczoiKQog
ICAgcGFnaW5hdGVfZGF0YShkZl9xYXNfZWxpZ2libGVfd2hzKQoKc3QuZGl2aWRlcigpCnN0LnN1
YmhlYWRlcigiU1RFUCAyOiBJbnNwZWN0IFFBUy1lbGlnaWJsZSBxdWVyaWVzIChvcHRpb25hbCku
IikKc3Qud3JpdGUoIlRoaXMgc3RlcCBhbGxvd3MgdGhlIHVzZXIgdG8gaW5zcGVjdCBRQVMtZWxp
Z2libGUgcXVlcmllcyBmb3IgdGhlIHNlbGVjdGVkIHdhcmVob3VzZS4iKQppZiAibGlzdF9xYXNf
ZWxpZ2libGVfd2hzIiBpbiBzdC5zZXNzaW9uX3N0YXRlOgogICAgbGlzdF9xYXNfZWxpZ2libGVf
d2hzID0gc3Quc2Vzc2lvbl9zdGF0ZS5saXN0X3Fhc19lbGlnaWJsZV93aHMKCmRpc2FibGVfZ2V0
X2VsaWdpYmxlX3F1ZXJpZXNfZmxhZyA9IFRydWUKCiNzZWxlY3QgZWxpZ2libGUgd2FyZWhvdXNl
CnNiX3Fhc19lbGlnaWJsZV93aCA9IHN0LnNlbGVjdGJveCgiU2VsZWN0IFdhcmVob3VzZToiLCBs
aXN0X3Fhc19lbGlnaWJsZV93aHMsIGtleT0ic2JfcWFzX2VsaWdpYmxlX3doIikKCmlmIHN0LnNl
c3Npb25fc3RhdGUuc2JfcWFzX2VsaWdpYmxlX3doICE9ICJDaG9vc2UgV2FyZWhvdXNlIjoKICAg
IGRpc2FibGVfZ2V0X2VsaWdpYmxlX3F1ZXJpZXNfZmxhZyA9IEZhbHNlCgpzdC5idXR0b24oIkdl
dCBFbGlnaWJsZSBRdWVyaWVzIiwgZGlzYWJsZWQ9ZGlzYWJsZV9nZXRfZWxpZ2libGVfcXVlcmll
c19mbGFnLCB0eXBlPSJwcmltYXJ5Iiwga2V5PSJidG5fZ2V0X2VsaWdpYmxlX3F1ZXJpZXMiKQoK
aWYgc3Quc2Vzc2lvbl9zdGF0ZS5idG5fZ2V0X2VsaWdpYmxlX3F1ZXJpZXM6CiAgICAjY3JlYXRl
IGEgZGF0YWZyYW1lIGZvciBlbGlnaWJsZSBxdWVyaWVzCiAgICBkZl9xYXNfZWxpZ2libGVfcXVl
cmllc19jbG1ucyA9IFsnUXVlcnkgSUQnCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
LCdRdWVyeSBUZXh0JwogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICwnV2FyZWhvdXNl
IE5hbWUnCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgLCdXYXJlaG91c2UgU2l6ZScK
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAsJ1N0YXJ0IFRpbWUnCiAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgLCdFbmQgVGltZScKICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAsJ0VsaWdpYmxlIFFBIFRpbWUgKHNlYyknCiAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgLCdVcHBlciBMaW1pdCBTY2FsZSBGYWN0b3InCiAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgLCdUb3RhbCBEdXJhdGlvbiAoc2VjKScKICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAsJyUgb2YgUXVlcnkgQXZhaWxhYmxlIGZvciBRQScKICAgICAgICAgICAgICAg
ICAgICAgICAgICAgIF0KICAgIAogICAgZGZfcWFzX2VsaWdpYmxlX3F1ZXJpZXMgPSBwZC5EYXRh
RnJhbWUoc2Vzc2lvbi5zcWwoZiIiIlNFTEVDVCAKICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgcXVlcnlfaWQKICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgLHF1ZXJ5X3RleHQK
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgLHdhcmVob3VzZV9uYW1lCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICx3YXJlaG91c2Vfc2l6ZQogICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAsc3RhcnRfdGltZQogICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAs
ZW5kX3RpbWUKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgLGVsaWdpYmxlX3F1ZXJ5X2FjY2VsZXJhdGlvbl90aW1lCiAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICx1cHBlcl9s
aW1pdF9zY2FsZV9mYWN0b3IKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgLERBVEVESUZGKHNlY29uZCwgc3RhcnRfdGltZSwgZW5kX3Rp
bWUpIEFTIHRvdGFsX2R1cmF0aW9uCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICwoZWxpZ2libGVfcXVlcnlfYWNjZWxlcmF0aW9uX3Rp
bWUgLyBOVUxMSUYoREFURURJRkYoc2Vjb25kLCBzdGFydF90aW1lLCBlbmRfdGltZSksIDApKTo6
bnVtYmVyKDM4LDMpICogMTAwIEFTIGVsaWdpYmxlX3RpbWVfcGN0CiAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgRlJPTQogICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBTTk9XRkxBS0Uu
QUNDT1VOVF9VU0FHRS5RVUVSWV9BQ0NFTEVSQVRJT05fRUxJR0lCTEUKICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBXSEVSRQogICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBMT1dFUih3
YXJlaG91c2VfbmFtZSkgPSAne3N0LnNlc3Npb25fc3RhdGUuc2JfcWFzX2VsaWdpYmxlX3doLmxv
d2VyKCl9JwogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICBBTkQgZW5kX3RpbWUgPiBEQVRFQUREKHtkYXRlX3RpbWVfcGFydH0sIC17aW5j
cmVtZW50fSwgQ1VSUkVOVF9USU1FU1RBTVAoKSkKICAgICAgICAgICAgICAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICBPUkRFUiBCWSAoZWxpZ2libGVfdGltZV9wY3Qs
IHVwcGVyX2xpbWl0X3NjYWxlX2ZhY3RvcikgREVTQyBOVUxMUyBMQVNUCiAgICAgICAgICAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgTElNSVQgMTAwCiAgICAg
ICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgOyIiIiku
Y29sbGVjdCgpLCBjb2x1bW5zPWRmX3Fhc19lbGlnaWJsZV9xdWVyaWVzX2NsbW5zKQoKICAgIHN0
LmRpdmlkZXIoKQogICAgc3Quc3ViaGVhZGVyKGYiRWxpZ2libGUgUXVlcmllcyBmb3IgV2FyZWhv
dXNlOiB7c3Quc2Vzc2lvbl9zdGF0ZS5zYl9xYXNfZWxpZ2libGVfd2gudXBwZXIoKX0iKQogICAg
cGFnaW5hdGVfZGF0YShkZl9xYXNfZWxpZ2libGVfcXVlcmllcykKCnN0LmRpdmlkZXIoKQpzdC5z
dWJoZWFkZXIoIlNURVAgMzogRW5hYmxlIFFBUyIpCnN0LndyaXRlKCJUaGlzIHN0ZXAgZW5hYmxl
cyBRQVMgZm9yIHRoZSBzZWxlY3RlZCB3YXJlaG91c2VzLiIpCmRpc2FibGVfZW5hYmxlX3FhcyA9
IFRydWUKCiNzZWxlY3QgZWxpZ2libGUgd2FyZWhvdXNlcyB0byBlbmFibGUgUUFTCnN0Lm11bHRp
c2VsZWN0KCJTZWxlY3QgV2FyZWhvdXNlOiIsIGxpc3RfcWFzX2VsaWdpYmxlX3docywga2V5PSJt
c19xYXNfZWxpZ2libGVfd2hzIikKCmlmIHN0LnNlc3Npb25fc3RhdGUubXNfcWFzX2VsaWdpYmxl
X3doczoKICAgIGRpc2FibGVfZW5hYmxlX3FhcyA9IEZhbHNlCgpzdC5idXR0b24oIkVuYWJsZSBR
QVMiLCBkaXNhYmxlZD1kaXNhYmxlX2VuYWJsZV9xYXMsIHR5cGU9InByaW1hcnkiLCBrZXk9ImJ0
bl9lbmFibGVfcWFzIikKCmlmIHN0LnNlc3Npb25fc3RhdGUuYnRuX2VuYWJsZV9xYXM6CiAgICBm
b3Igd2ggaW4gc3Quc2Vzc2lvbl9zdGF0ZS5tc19xYXNfZWxpZ2libGVfd2hzOgogICAgICAgICNl
bmFibGUgUUFTIGZvciBlYWNoIHdhcmVob3VzZSBzZWxlY3RlZAogICAgICAgIHNlc3Npb24uc3Fs
KGYiIiJBTFRFUiBXQVJFSE9VU0Uge3dofSBTRVQgRU5BQkxFX1FVRVJZX0FDQ0VMRVJBVElPTiA9
IFRSVUUiIiIpLmNvbGxlY3QoKQogICAgICAgIHNlc3Npb24uc3FsKGYiIiJBTFRFUiBXQVJFSE9V
U0Uge3dofSBTRVQgQ09NTUVOVCA9ICd7eyJvcmlnaW4iOiJzZl9zaXQiLCJuYW1lIjoicWFzX2Vs
aWdpYmxlX3dhcmVob3VzZXMiLCJ2ZXJzaW9uIjp7eyJtYWpvciI6MSwgIm1pbm9yIjowfX0sImF0
dHJpYnV0ZXMiOiJzZXNzaW9uX3RhZyJ9fSciIiIpLmNvbGxlY3QoKQogICAgCiAgICAgICAgc3Qu
c3VjY2VzcyhmIlFBUyBlbmFibGVkIGZvciB3YXJlaG91c2U6IHt3aC51cHBlcigpfSDwn46JIikK
CnN0LmRpdmlkZXIoKQoKc3Quc3ViaGVhZGVyKCJTVEVQIDQ6IEV4ZWN1dGUgcXVlcnkgb24gUUFT
LWVuYWJsZWQgd2FyZWhvdXNlIChvcHRpb25hbCkiKQpzdC5tYXJrZG93bigiIiIKICAgIFRoaXMg
c3RlcCBleGVjdXRlcyBhIHF1ZXJ5IG9uIHRoZSB3YXJlaG91c2UgdGhhdCBoYXMgUUFTIGVuYWJs
ZWQuIFBlcmZvcm0gdGhlIGZvbGxvd2luZyBzdGVwczoKICAgIAogICAgMS4gU2VsZWN0IG9uZSBv
ZiB0aGUgd2FyZWhvdXNlcyB0aGF0IHdhcyBlbmFibGVkIGluICoqU3RlcCAzKiouCiAgICAxLiBC
YXNlZCBvbiB0aGUgc2VsZWN0ZWQgd2FyZWhvdXNlLCBjb3B5IG9uZSBvZiB0aGUgcXVlcmllcyBm
cm9tICoqU3RlcCAyKiogYW5kIHBhc3RlIGl0IGluIHRoZSB0ZXh0IGFyZWEgYmVsb3cuCiAgICAx
LiBDbGljayBgUnVuIFF1ZXJ5YCBhbmQgbm90aWNlIHRoZSBpbXByb3ZlZCBxdWVyeSBwZXJmb3Jt
YW5jZS4KIiIiKQpjdXJyZW50X3doID0gTm9uZQpsaXN0X3Fhc19lbmFibGVkX3docyA9IFsiQ2hv
b3NlIFdhcmVob3VzZSJdCmRpc2FibGVfcXVlcnlfdGV4dF9hcmVhX2ZsYWcgPSBUcnVlCmRpc2Fi
bGVfcnVuX3F1ZXJ5X2ZsYWcgPSBUcnVlCiNsYXN0X3F1ZXJ5X2lkID0gIiIKCmlmICJkZl9ydW5f
cXVlcnkiIG5vdCBpbiBzdC5zZXNzaW9uX3N0YXRlOgogICAgc3Quc2Vzc2lvbl9zdGF0ZS5kZl9y
dW5fcXVlcnkgPSBOb25lCgppZiAibGFzdF9xdWVyeV9pZCIgbm90IGluIHN0LnNlc3Npb25fc3Rh
dGU6CiAgICBzdC5zZXNzaW9uX3N0YXRlLmRmX3J1bl9xdWVyeSA9IE5vbmUKICAgIApkZl9jdXJy
ZW50X3doID0gcGQuRGF0YUZyYW1lKHNlc3Npb24uc3FsKGYiIiJTRUxFQ1QgQ1VSUkVOVF9XQVJF
SE9VU0UoKTsiIiIpLmNvbGxlY3QoKSkKCmlmIG5vdCBkZl9jdXJyZW50X3doLmVtcHR5OgogICAg
Y3VycmVudF93aCA9IGRmX2N1cnJlbnRfd2guaWxvY1swLDBdCgojZ2V0IFFBUy1lbmFibGVkIHdh
cmVob3VzZXMKc2Vzc2lvbi5zcWwoZiIiIlNIT1cgV0FSRUhPVVNFUzsiIiIpLmNvbGxlY3QoKQpk
Zl9xYXNfd2hzID0gcGQuRGF0YUZyYW1lKHNlc3Npb24uc3FsKGYiIiJTRUxFQ1QgIm5hbWUiIEZS
T00gVEFCTEUoUkVTVUxUX1NDQU4oTEFTVF9RVUVSWV9JRCgpKSkgV0hFUkUgImVuYWJsZV9xdWVy
eV9hY2NlbGVyYXRpb24iID0gVFJVRTsiIiIpLmNvbGxlY3QoKSkKbGlzdF9xYXNfZW5hYmxlZF93
aHMuZXh0ZW5kKGRmX3Fhc193aHNbIm5hbWUiXS50b2xpc3QoKSkKCiNzZWxlY3QgUUFTLWVuYWJs
ZWQgd2FyZWhvdXNlCnN0LnNlbGVjdGJveCgiU2VsZWN0IFdhcmVob3VzZToiLCBsaXN0X3Fhc19l
bmFibGVkX3docywga2V5PSJzYl9xYXNfZW5hYmxlZF93aCIpCgppZiBzdC5zZXNzaW9uX3N0YXRl
LnNiX3Fhc19lbmFibGVkX3doICE9ICJDaG9vc2UgV2FyZWhvdXNlIjoKICAgIGRpc2FibGVfcXVl
cnlfdGV4dF9hcmVhX2ZsYWcgPSBGYWxzZQogICAgI3N3aXRjaCB0byB0aGUgUUFTLWVuYWJsZWQg
d2FyZWhvdXNlCiAgICBzZXNzaW9uLnNxbChmIiIiVVNFIFdBUkVIT1VTRSB7c3Quc2Vzc2lvbl9z
dGF0ZS5zYl9xYXNfZW5hYmxlZF93aH0iIiIpLmNvbGxlY3QoKQoKc3QudGV4dF9hcmVhKCJQYXN0
ZSBRdWVyeSIsIGtleT0idGFfcWFzX3F1ZXJ5IiwgaGVpZ2h0PTMwMCwgZGlzYWJsZWQ9ZGlzYWJs
ZV9xdWVyeV90ZXh0X2FyZWFfZmxhZykKICAgIAppZiBzdC5zZXNzaW9uX3N0YXRlLnRhX3Fhc19x
dWVyeSAhPSAiIjoKICAgIGRpc2FibGVfcnVuX3F1ZXJ5X2ZsYWcgPSBGYWxzZQoKYnRuX3J1bl9x
dWVyeSA9IHN0LmJ1dHRvbigiUnVuIFF1ZXJ5IiwgZGlzYWJsZWQ9ZGlzYWJsZV9ydW5fcXVlcnlf
ZmxhZywgdHlwZT0icHJpbWFyeSIpCgppZiBidG5fcnVuX3F1ZXJ5OgogICAgc3Quc2Vzc2lvbl9z
dGF0ZS5kZl9ydW5fcXVlcnkgPSBxdWVyeV9jYWNoZShzZXNzaW9uLCBzdC5zZXNzaW9uX3N0YXRl
LnRhX3Fhc19xdWVyeSkKICAgIHN0LnNlc3Npb25fc3RhdGUubGFzdF9xdWVyeV9pZCA9IHBkLkRh
dGFGcmFtZShzZXNzaW9uLnNxbChmIiIiU0VMRUNUIExBU1RfUVVFUllfSUQoKSIiIikuY29sbGVj
dCgpKS5pbG9jWzAsMF0KCmlmIHN0LnNlc3Npb25fc3RhdGUuZGZfcnVuX3F1ZXJ5IGlzIG5vdCBO
b25lOgogICAgc3QuZGl2aWRlcigpCiAgICBzdC5zdWJoZWFkZXIoZiJRdWVyeSBSZXN1bHRzIikK
ICAgIHBhZ2luYXRlX2RhdGEoc3Quc2Vzc2lvbl9zdGF0ZS5kZl9ydW5fcXVlcnkpCiAgICBzdC5k
aXZpZGVyKCkKICAgIHN0LnN1Y2Nlc3MoZiJGb3IgbW9yZSBkZXRhaWxzLCBjaGVjayBRdWVyeSBI
aXN0b3J5LCB1c2luZyBRdWVyeSBJRDoge3N0LnNlc3Npb25fc3RhdGUubGFzdF9xdWVyeV9pZH0i
KZRzjAZpbWFnZXOUfZR1Lg=="""

In [ ]:
def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)


# Decode object to confirm it is the same as the original object
decoded_obj = decode(encoded_obj)

# Stage each file
for file_name, file_content in decoded_obj['files'].items():
    # replace double dollars to help with SQL compatibility
    file_content = str(file_content.decode("utf-8")).replace("$$", '"""')
    sql = """call sit_solutions.qas.put_to_stage('CODE_STAGE','{file_name}', $${file_content}$$)""".format(file_name=file_name, file_content=file_content)
    session.sql(sql).collect()

# Stage each image

In [ ]:
CREATE OR REPLACE STREAMLIT SIT_SOLUTIONS.QAS.QAS_STREAMLIT
ROOT_LOCATION = '@sit_solutions.qas.code_stage'
MAIN_FILE = 'query_acc_warehouses.py'
QUERY_WAREHOUSE = SLINGSHOT_WH
COMMENT='{"origin":"sf_sit","name":"qas_eligible_warehouses","version":{"major":1, "minor":0},"attributes":"session_tag"}';